**Réseau de Neurones Convolutif pour la Détection et le Diagnostic des Tumeurs Cérébrales 
(PyTorch, F1-score : 0,97)**

In [ ]:
!pip install split-folders
!pip install torch-summary

In [ ]:
# # Importation des bibliothèques essentielles
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style='darkgrid')
import copy # Un module qui fournit des fonctions pour créer des copies d'objets, utile pour éviter les modifications involontaires des variables.
import os 
import torch
from PIL import Image
from torch.utils.data import Dataset 
import torchvision
import torchvision.transforms as transforms # Un module de la bibliothèque torchvision qui fournit des transformations d'images courantes, telles que le redimensionnement, le recadrage et la normalisation.
from torch.utils.data import random_split 
from torch.optim.lr_scheduler import ReduceLROnPlateau ## Un planificateur PyTorch qui ajuste le taux d'apprentissage pendant l'entraînement en fonction d'une métrique spécifiée, en le réduisant lorsque cette métrique atteint un plateau.
import torch.nn as nn # Un module de PyTorch qui fournit des classes pour définir et construire des réseaux de neurones.
from torchvision import utils 
from torchvision.datasets import ImageFolder
import splitfolders
from torchsummary import summary
import torch.nn.functional as F
import pathlib
from sklearn.metrics import confusion_matrix, classification_report
import itertools 
from tqdm.notebook import trange, tqdm 
from torch import optim
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from IPython.core.display import display, HTML, Javascript

color_map = ['#FFFFFF','#FF5733']

prompt = color_map[-1]
main_color = color_map[0]
strong_main_color = color_map[1]
custom_colors = [strong_main_color, main_color]

css_file = '''
div #notebook {
background-color: white;
line-height: 20px;
}

#notebook-container {
%s
margin-top: 2em;
padding-top: 2em;
border-top: 4px solid %s;
-webkit-box-shadow: 0px 0px 8px 2px rgba(224, 212, 226, 0.5);
    box-shadow: 0px 0px 8px 2px rgba(224, 212, 226, 0.5);
}

div .input {
margin-bottom: 1em;
}

.rendered_html h1, .rendered_html h2, .rendered_html h3, .rendered_html h4, .rendered_html h5, .rendered_html h6 {
color: %s;
font-weight: 600;
}

div.input_area {
border: none;
    background-color: %s;
    border-top: 2px solid %s;
}

div.input_prompt {
color: %s;
}

div.output_prompt {
color: %s; 
}

div.cell.selected:before, div.cell.selected.jupyter-soft-selected:before {
background: %s;
}

div.cell.selected, div.cell.selected.jupyter-soft-selected {
    border-color: %s;
}

.edit_mode div.cell.selected:before {
background: %s;
}

.edit_mode div.cell.selected {
border-color: %s;

}
'''

def to_rgb(h): 
    return tuple(int(h[i:i+2], 16) for i in [0, 2, 4])

main_color_rgba = 'rgba(%s, %s, %s, 0.1)' % (to_rgb(main_color[1:]))
open('notebook.css', 'w').write(css_file % ('width: 95%;', main_color, main_color, main_color_rgba, 
                                            main_color,  main_color, prompt, main_color, main_color, 
                                            main_color, main_color))

def nb(): 
    return HTML("<style>" + open("notebook.css", "r").read() + "</style>")
nb()


## <b>1 <span style='color:#e61227'>|</span> Introduction</b> 

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px;background-image: url(https://i.imgur.com/GVd0La1.png)">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#e61227'>1.1 |</span></b> Pourquoi avons-nous besoin de cette étude ?</b></p>
</div>

**L'apprentissage profond** est devenu un outil puissant dans le domaine de **l'imagerie médicale** et a montré un grand potentiel pour aider la communauté médicale dans la détection et le diagnostic des **tumeurs cérébrales**. En utilisant des algorithmes d'apprentissage profond, nous pouvons analyser des images médicales, telles que des **IRM** ou des **scanners CT**, avec une précision et une efficacité sans précédent. De plus, il peut aider à la classification des tumeurs cérébrales en différents sous-types. En entraînant des modèles sur de grands ensembles de données d'images de tumeurs cérébrales étiquetées, les algorithmes d'apprentissage profond peuvent apprendre à distinguer les différents types de tumeurs, tels que les gliomes, les méningiomes ou les tumeurs métastatiques. Cette capacité de classification peut aider à déterminer l'approche thérapeutique et le pronostic appropriés pour les patients.

Dans l'ensemble, l'apprentissage profond a le potentiel de révolutionner la détection et le diagnostic des tumeurs cérébrales. En exploitant la puissance des réseaux de neurones, nous pouvons améliorer la précision, l'efficacité et la compréhension des images des tumeurs cérébrales, ce qui conduira finalement à de meilleurs soins et résultats pour les patients dans le domaine de la neuro-oncologie.

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px;background-image: url(https://i.imgur.com/GVd0La1.png)">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#e61227'>1.2 |</span></b> Énoncé du problème</b></p>
</div>

La détection et la classification précises des **tumeurs cérébrales** jouent un rôle crucial dans le diagnostic et la planification du traitement des patients. Cependant, l'interprétation manuelle des **images médicales**, telles que les IRM, peut être longue et subjective, ce qui peut entraîner des erreurs et des retards dans les soins aux patients. Par conséquent, il existe un besoin d'une méthode automatisée et fiable pour détecter et classifier les tumeurs cérébrales à partir des images médicales.

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px;background-image: url(https://i.imgur.com/GVd0La1.png)">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#e61227'>1.3 |</span></b> Objectif de l'étude</b></p>
</div>

Cette étude vise à développer un Réseau de Neurones Convolutif **(CNN)** en utilisant le cadre **PyTorch** capable de détecter et de classifier avec précision les **tumeurs cérébrales** à partir des IRM. Le CNN sera entraîné sur un grand ensemble de données d'images de tumeurs cérébrales étiquetées pour apprendre les motifs et les caractéristiques associés aux différents types de tumeurs. L'objectif de l'étude est d'atteindre une haute précision dans la détection et la classification des tumeurs, offrant ainsi un outil précieux pour les professionnels de la santé dans le domaine de la neuro-oncologie. Le but ultime est d'améliorer l'efficacité et la précision du diagnostic des tumeurs cérébrales, permettant ainsi une planification de traitement appropriée et en temps voulu pour les patients.

## <b>2 <span style='color:#e61227'>|</span> Dataset</b> 

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px;background-image: url(https://i.imgur.com/GVd0La1.png)">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#e61227'>2.1 |</span></b> Load Dataset</b></p>
</div>

In [ ]:
labels_df = pd.read_csv('/kaggle/input/brian-tumor-dataset/metadata.csv')
print(labels_df.head().to_markdown())

In [ ]:
os.listdir('/kaggle/input/brian-tumor-dataset/Brain Tumor Data Set/Brain Tumor Data Set')

In [ ]:
labels_df.shape

## <b>3 <span style='color:#e61227'>|</span> Data Preparation </b> 

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px;background-image: url(https://i.imgur.com/GVd0La1.png)">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#e61227'>3.1 |</span></b> Splitting Dataset</b></p>
</div>

- Nous devons évaluer le modèle sur les ensembles de validation afin de suivre la performance du modèle pendant l'entraînement. Ensuite, utilisons 20 % de l'ensemble de données pour l'**ensemble de validation** et le reste pour l'**ensemble d'entraînement**, afin d'obtenir une répartition **80/20** !

In [ ]:
# Chemin du Dataset
data_dir = '/kaggle/input/brian-tumor-dataset/Brain Tumor Data Set/Brain Tumor Data Set'
data_dir = pathlib.Path(data_dir)

# Diviser l'ensemble de données en ensembles d'entraînement, de validation et de test
splitfolders.ratio(data_dir, output='brain', seed=20, ratio=(0.8, 0.2))


# Nouveau chemin du dataset 
data_dir = '/kaggle/working/brain'
data_dir = pathlib.Path(data_dir)

## <b>4 <span style='color:#e61227'>|</span> Image Augmentation Definitions</b> 

In [ ]:
# definir transformation
transform = transforms.Compose(
    [
        transforms.Resize((256,256)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225])
   ]
)

In [ ]:
# Définir un objet de jeu de données personnalisé pour l'entraînement et la validation.
train_set = torchvision.datasets.ImageFolder(data_dir.joinpath("train"), transform=transform) 
train_set.transform
val_set = torchvision.datasets.ImageFolder(data_dir.joinpath("val"), transform=transform)
val_set.transform

In [ ]:
# Visualiser quelques images de l'ensemble d'entraînement
CLA_label = {
    0 : 'Brain Tumor',
    1 : 'Healthy'
} 
figure = plt.figure(figsize=(10, 10))
cols, rows = 4, 4
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(train_set), size=(1,)).item()
    img, label = train_set[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(CLA_label[label])
    plt.axis("off")
    img_np = img.numpy().transpose((1, 2, 0))
    #Clipper les valeurs des pixels dans la plage [0, 1]
    img_valid_range = np.clip(img_np, 0, 1)
    plt.imshow(img_valid_range)
    plt.suptitle('Brain Images', y=0.95)
plt.show()

## <b>5 <span style='color:#e61227'>|</span> Creating Dataloaders</b> 

In [ ]:
# import and load train, validation
batch_size = 64

train_loader = torch.utils.data.DataLoader(train_set, batch_size = batch_size, shuffle = True, num_workers = 2)
val_loader = torch.utils.data.DataLoader(val_set, batch_size = batch_size, shuffle = True, num_workers = 2)


In [ ]:
# "imprimer la forme des données d'entraînement et de validation"
for key, value in {'Training data': train_loader, "Validation data": val_loader}.items():
    for X, y in value:
        print(f"{key}:")
        print(f"Shape of X : {X.shape}")
        print(f"Shape of y: {y.shape} {y.dtype}\n")
        break

## <b>6 <span style='color:#e61227'>|</span> Define Brain Tumor Classifier</b> 


In [ ]:
''''Cette fonction peut être utile pour déterminer la taille de sortie d'une couche convolutionnelle dans un réseau de neurones, étant donné les dimensions de l'entrée et les paramètres de la couche convolutionnelle.'''

def findConv2dOutShape(hin,win,conv,pool=2):
    # get conv arguments
    kernel_size = conv.kernel_size
    stride=conv.stride
    padding=conv.padding
    dilation=conv.dilation

    hout=np.floor((hin+2*padding[0]-dilation[0]*(kernel_size[0]-1)-1)/stride[0]+1)
    wout=np.floor((win+2*padding[1]-dilation[1]*(kernel_size[1]-1)-1)/stride[1]+1)

    if pool:
        hout/=pool
        wout/=pool
    return int(hout),int(wout)

In [ ]:
#Définir l'architecture du modèle CNN_TUMOR
class CNN_TUMOR(nn.Module):
    
    # Initialisation du réseau
    def __init__(self, params):
        
        super(CNN_TUMOR, self).__init__()
    
        Cin,Hin,Win = params["shape_in"]
        init_f = params["initial_filters"] 
        num_fc1 = params["num_fc1"]  
        num_classes = params["num_classes"] 
        self.dropout_rate = params["dropout_rate"] 
        
        # Les couches de convolution
        self.conv1 = nn.Conv2d(Cin, init_f, kernel_size=3)
        h,w=findConv2dOutShape(Hin,Win,self.conv1)
        self.conv2 = nn.Conv2d(init_f, 2*init_f, kernel_size=3)
        h,w=findConv2dOutShape(h,w,self.conv2)
        self.conv3 = nn.Conv2d(2*init_f, 4*init_f, kernel_size=3)
        h,w=findConv2dOutShape(h,w,self.conv3)
        self.conv4 = nn.Conv2d(4*init_f, 8*init_f, kernel_size=3)
        h,w=findConv2dOutShape(h,w,self.conv4)
        
        # Calculer la taille aplatie
        self.num_flatten=h*w*8*init_f
        self.fc1 = nn.Linear(self.num_flatten, num_fc1)
        self.fc2 = nn.Linear(num_fc1, num_classes)

    def forward(self,X):
        
        # Convolution et couches de pooling
        X = F.relu(self.conv1(X)); 
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv3(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv4(X))
        X = F.max_pool2d(X, 2, 2)
        X = X.view(-1, self.num_flatten)
        X = F.relu(self.fc1(X))
        X = F.dropout(X, self.dropout_rate)
        X = self.fc2(X)
        return F.log_softmax(X, dim=1)

In [ ]:
params_model={
        "shape_in": (3,256,256), 
        "initial_filters": 8,    
        "num_fc1": 100,
        "dropout_rate": 0.25,
        "num_classes": 2}

# Créer une instance de la classe Network
cnn_model = CNN_TUMOR(params_model)

# Définir l'approche matérielle de calcul (GPU/CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = cnn_model.to(device)

In [ ]:
# Résumé du modèle pour le modèle CNN
summary(cnn_model, input_size=(3, 256, 256),device=device.type)

## <b>7 <span style='color:#e61227'>|</span> Loss Function Definition</b> 

In [ ]:
loss_func = nn.NLLLoss(reduction="sum")

## <b>8 <span style='color:#e61227'>|</span> Optimiser Definition</b> 

In [ ]:
opt = optim.Adam(cnn_model.parameters(), lr=3e-4)
lr_scheduler = ReduceLROnPlateau(opt, mode='min',factor=0.5, patience=20,verbose=1)

## <b>9 <span style='color:#e61227'>|</span> Training Model</b> 

In [ ]:
# Fonction pour obtenir le taux d'apprentissage
def get_lr(opt):
    for param_group in opt.param_groups:
        return param_group['lr']

# Fonction pour calculer la valeur de la perte par lot de données
def loss_batch(loss_func, output, target, opt=None):
    
    loss = loss_func(output, target) # get loss
    pred = output.argmax(dim=1, keepdim=True) # Get Output Class
    metric_b=pred.eq(target.view_as(pred)).sum().item() # get performance metric
    
    if opt is not None:
        opt.zero_grad()
        loss.backward()
        opt.step()

    return loss.item(), metric_b

# Calculer la valeur de la perte et les métriques de performance pour l'ensemble du dataset (époque)
def loss_epoch(model,loss_func,dataset_dl,opt=None):
    
    run_loss=0.0 
    t_metric=0.0
    len_data=len(dataset_dl.dataset)

    # boucle interne sur le dataset
    for xb, yb in dataset_dl:
        # Déplacer le lot vers le périphérique
        xb=xb.to(device)
        yb=yb.to(device)
        output=model(xb) # Obtenir la sortie du modèle
        loss_b,metric_b=loss_batch(loss_func, output, yb, opt) # Obtenir la perte par lot
        run_loss+=loss_b        # Mettre à jour la perte cumulative

        if metric_b is not None: # Mettre à jour la métrique en cours
            t_metric+=metric_b    
    
    loss=run_loss/float(len_data)  # Valeur de la perte moyenne
    metric=t_metric/float(len_data) # Valeur moyenne de la métrique
    
    return loss, metric

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px;background-image: url(https://i.imgur.com/GVd0La1.png)">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#e61227'>9.2 |</span></b> Training Function</b></p>
</div>


In [ ]:
def Train_Val(model, params,verbose=False):
    
    # Obtenir les paramètres
    epochs=params["epochs"]
    loss_func=params["f_loss"]
    opt=params["optimiser"]
    train_dl=params["train"]
    val_dl=params["val"]
    lr_scheduler=params["lr_change"]
    weight_path=params["weight_path"]
    
    # Historique des valeurs de perte à chaque époque
    loss_history={"train": [],"val": []} 
    # Historique des valeurs des métriques à chaque époque
    metric_history={"train": [],"val": []} 
    # Une copie profonde des poids pour le meilleur modèle performant
    best_model_wts = copy.deepcopy(model.state_dict()) 
    # Initialiser la meilleure perte à une valeur élevée
    best_loss=float('inf') 

# Entraîner le modèle pendant n_epochs (la progression de l'entraînement en imprimant le numéro de l'époque et le taux d'apprentissage associé.) 
    
    for epoch in tqdm(range(epochs)):
        
        # Obtenir le taux d'apprentissage
        current_lr=get_lr(opt)
        if(verbose):
            print('Epoch {}/{}, current lr={}'.format(epoch, epochs - 1, current_lr))

        
# Le processus d'entraînement du modèle

        
        model.train()
        train_loss, train_metric = loss_epoch(model,loss_func,train_dl,opt)

        # collecter les pertes
        loss_history["train"].append(train_loss)
        metric_history["train"].append(train_metric)
        

# Évaluer le processus du modèle

        
        model.eval()
        with torch.no_grad():
            val_loss, val_metric = loss_epoch(model,loss_func,val_dl)
        
        # Stocker le meilleur modèle
        if(val_loss < best_loss):
            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            
            # Stocker les poids dans un fichier local
            torch.save(model.state_dict(), weight_path)
            if(verbose):
                print("Copied best model weights!")
        
        # Collecter la perte et la métrique pour le jeu de données de validation
        loss_history["val"].append(val_loss)
        metric_history["val"].append(val_metric)
        
        # Planification du taux d'apprentissage
        lr_scheduler.step(val_loss)
        if current_lr != get_lr(opt):
            if(verbose):
                print("Loading best model weights!")
            model.load_state_dict(best_model_wts) 

        if(verbose):
            print(f"train loss: {train_loss:.6f}, dev loss: {val_loss:.6f}, accuracy: {100*val_metric:.2f}")
            print("-"*10) 

    # Charger les poids du meilleur modèle
    model.load_state_dict(best_model_wts)
        
    return model, loss_history, metric_history

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px;background-image: url(https://i.imgur.com/GVd0La1.png)">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#e61227'>9.3 |</span></b> Training Process </b></p>
</div>

In [ ]:
# Définir divers paramètres utilisés pour l'entraînement et l'évaluation d'un modèle CNN

params_train={
 "train": train_loader,"val": val_loader,
 "epochs": 60,
 "optimiser": optim.Adam(cnn_model.parameters(),lr=3e-4),
 "lr_change": ReduceLROnPlateau(opt,
                                mode='min',
                                factor=0.5,
                                patience=20,
                                verbose=0),
 "f_loss": nn.NLLLoss(reduction="sum"),
 "weight_path": "weights.pt",
}

# Entraîner et valider le modèle 
cnn_model,loss_hist,metric_hist = Train_Val(cnn_model,params_train)

## <b>10 <span style='color:#e61227'>|</span> Evaluation Metric Visualization </b> 

In [ ]:
# Tracé de l'historique de convergence
epochs=params_train["epochs"]
fig,ax = plt.subplots(1,2,figsize=(12,5))

sns.lineplot(x=[*range(1,epochs+1)],y=loss_hist["train"],ax=ax[0],label='loss_hist["train"]')
sns.lineplot(x=[*range(1,epochs+1)],y=loss_hist["val"],ax=ax[0],label='loss_hist["val"]')
sns.lineplot(x=[*range(1,epochs+1)],y=metric_hist["train"],ax=ax[1],label='Acc_hist["train"]')
sns.lineplot(x=[*range(1,epochs+1)],y=metric_hist["val"],ax=ax[1],label='Acc_hist["val"]')

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px;background-image: url(https://i.imgur.com/GVd0La1.png)">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#e61227'>10.2 |</span></b> Confusion_Matrix </b></p>
</div>

In [ ]:
# Définir une fonction pour le rapport de classification
def Ture_and_Pred(val_loader, model):
    i = 0
    y_true = []
    y_pred = []
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.numpy()
        outputs = model(images)
        _, pred = torch.max(outputs.data, 1)
        pred = pred.detach().cpu().numpy()
        
        y_true = np.append(y_true, labels)
        y_pred = np.append(y_pred, pred)
    
    return y_true, y_pred


# Vérifier la matrice de confusion pour l'analyse des erreurs
y_true, y_pred = Ture_and_Pred(val_loader, cnn_model)

print(classification_report(y_true, y_pred), '\n\n')
cm = confusion_matrix(y_true, y_pred)

In [ ]:
# Fonction de tracé de la matrice de confusion
def show_confusion_matrix(cm, CLA_label, title='Confusion matrix', cmap=plt.cm.YlGnBu):
    
    plt.figure(figsize=(10,7))
    plt.grid(False)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(CLA_label))

    plt.xticks(tick_marks, [f"{value}={key}" for key , value in CLA_label.items()], rotation=45)
    plt.yticks(tick_marks, [f"{value}={key}" for key , value in CLA_label.items()])

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, f"{cm[i,j]}\n{cm[i,j]/np.sum(cm)*100:.2f}%", horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.tight_layout()
    plt.show()

show_confusion_matrix(cm, CLA_label)

## <b>11<span style='color:#e61227'>|</span> Save Model </b>

In [ ]:
torch.save(cnn_model, "Brain_Tumor_model.pt")